# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.4/9545/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

['/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-01.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-02.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-03.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-04.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-05.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-06.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-07.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-08.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-09.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-10.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-11.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-12.csv',
 '/home/runner/work/dask-exa

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,983,Alice,0.3159072153158986,-0.019673773487784985
2000-01-01 00:00:01,1028,George,-0.17201993860603482,0.43595666389154153
2000-01-01 00:00:02,1003,Ray,0.03132347831613913,-0.17519522354244654
2000-01-01 00:00:03,1029,Quinn,0.7912068496836833,0.3455184184379485
2000-01-01 00:00:04,1055,Wendy,-0.6783800828260322,-0.08055769598189011
2000-01-01 00:00:05,1008,Yvonne,0.41658840814612597,0.4289141087944861
2000-01-01 00:00:06,1013,Patricia,0.9898160755802157,0.6238699043188576
2000-01-01 00:00:07,1001,Xavier,-0.98752514033202,0.508534697597558
2000-01-01 00:00:08,1037,Oliver,-0.05539054017067646,-0.1779060630115128


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1052,Ursula,-0.5533746445620105,-0.3865923936274107
2000-01-30 00:00:01,973,George,0.12810434864066855,0.47531642255423545
2000-01-30 00:00:02,929,Kevin,0.8923702108529377,-0.5766686397997691
2000-01-30 00:00:03,1000,George,-0.6777208268490023,0.8097251374505292
2000-01-30 00:00:04,950,Patricia,0.46743145498262995,0.5580505037473125
2000-01-30 00:00:05,997,Alice,-0.25795551725617005,0.6401811093336882
2000-01-30 00:00:06,967,Victor,-0.5462319941084401,0.3619887231784049
2000-01-30 00:00:07,1031,Edith,0.4057591166605128,0.17570800255952612
2000-01-30 00:00:08,997,George,-0.8849210368803704,-0.37584986296062906


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,983,Alice,0.315907,-0.019674
1,2000-01-01 00:00:01,1028,George,-0.172020,0.435957
2,2000-01-01 00:00:02,1003,Ray,0.031323,-0.175195
3,2000-01-01 00:00:03,1029,Quinn,0.791207,0.345518
4,2000-01-01 00:00:04,1055,Wendy,-0.678380,-0.080558


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,983,Alice,0.315907,-0.019674
1,2000-01-01 00:00:01,1028,George,-0.172020,0.435957
2,2000-01-01 00:00:02,1003,Ray,0.031323,-0.175195
3,2000-01-01 00:00:03,1029,Quinn,0.791207,0.345518
4,2000-01-01 00:00:04,1055,Wendy,-0.678380,-0.080558


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.7 s, sys: 426 ms, total: 4.13 s
Wall time: 2.85 s


name
Alice      -0.000539
Bob        -0.000356
Charlie    -0.000817
Dan         0.001951
Edith       0.000476
Frank       0.000292
George     -0.000412
Hannah     -0.001591
Ingrid      0.001535
Jerry      -0.001157
Kevin       0.002653
Laura      -0.002542
Michael    -0.000321
Norbert    -0.000050
Oliver     -0.001216
Patricia    0.000744
Quinn      -0.001521
Ray        -0.000091
Sarah       0.002263
Tim        -0.003071
Ursula      0.002541
Victor     -0.000899
Wendy       0.001047
Xavier     -0.002915
Yvonne      0.001254
Zelda       0.000945
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.12 s, sys: 118 ms, total: 1.23 s
Wall time: 905 ms


name
Alice      -0.000539
Bob        -0.000356
Charlie    -0.000817
Dan         0.001951
Edith       0.000476
Frank       0.000292
George     -0.000412
Hannah     -0.001591
Ingrid      0.001535
Jerry      -0.001157
Kevin       0.002653
Laura      -0.002542
Michael    -0.000321
Norbert    -0.000050
Oliver     -0.001216
Patricia    0.000744
Quinn      -0.001521
Ray        -0.000091
Sarah       0.002263
Tim        -0.003071
Ursula      0.002541
Victor     -0.000899
Wendy       0.001047
Xavier     -0.002915
Yvonne      0.001254
Zelda       0.000945
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.09 s, sys: 84.3 ms, total: 1.17 s
Wall time: 909 ms


name
Alice      -0.000539
Bob        -0.000356
Charlie    -0.000817
Dan         0.001951
Edith       0.000476
Frank       0.000292
George     -0.000412
Hannah     -0.001591
Ingrid      0.001535
Jerry      -0.001157
Kevin       0.002653
Laura      -0.002542
Michael    -0.000321
Norbert    -0.000050
Oliver     -0.001216
Patricia    0.000744
Quinn      -0.001521
Ray        -0.000091
Sarah       0.002263
Tim        -0.003071
Ursula      0.002541
Victor     -0.000899
Wendy       0.001047
Xavier     -0.002915
Yvonne      0.001254
Zelda       0.000945
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html